<a href="https://colab.research.google.com/github/chetna-17/Image-caption-generation-LLava/blob/main/useModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets peft torch torchvision

In [3]:
!pip install transformers accelerate bitsandbytes torchvision PIL

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [4]:
!pip install -U "transformers>=4.39.0"
!pip install peft bitsandbytes
!pip install -U "trl>=0.8.3"
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 108.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
  Using cached bitsandbytes-0.45.3-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.3 MB/s eta 0:00:00


In [5]:
import torch
from transformers import AutoTokenizer, AutoProcessor, TrainingArguments, LlavaForConditionalGeneration, BitsAndBytesConfig
from trl import SFTTrainer
from peft import LoraConfig

code for image uploading form google drive:

In [ ]:
model_id = "llava-hf/llava-1.5-7b-hf"

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

In [ ]:
model = LlavaForConditionalGeneration.from_pretrained(model_id,
                                                      quantization_config=quantization_config,
                                                      torch_dtype=torch.float16)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
LLAVA_CHAT_TEMPLATE = """A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. {% for message in messages %}{% if message['role'] == 'user' %}USER: {% else %}ASSISTANT: {% endif %}{% for item in message['content'] %}{% if item['type'] == 'text' %}{{ item['text'] }}{% elif item['type'] == 'image' %}<image>{% endif %}{% endfor %}{% if message['role'] == 'user' %} {% else %}{{eos_token}}{% endif %}{% endfor %}"""

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.chat_template = LLAVA_CHAT_TEMPLATE
processor = AutoProcessor.from_pretrained(model_id)
processor.tokenizer = tokenizer

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from PIL import Image
import torch
from transformers import AutoProcessor

def generate_image_caption(image_path, model, processor, tokenizer, max_new_tokens=100, device=None):

    # Load and process the image
    image = Image.open(image_path).convert("RGB")
    text_prompt = "Describe the <image>"
    processed_inputs = processor(images=image, text=text_prompt, return_tensors="pt")

    # Add the <image> token to the tokenizer's vocabulary if it doesn't exist
    if "<image>" not in tokenizer.get_vocab():
        tokenizer.add_tokens(["<image>"], special_tokens=True)

    # Reinitialize the processor with the updated tokenizer
    processor.tokenizer = tokenizer

    # Set device
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move model and inputs to the device
    model = model.to(device)
    processed_inputs = {k: v.to(device) for k, v in processed_inputs.items()}

    # Generate the caption
    outputs = model.generate(**processed_inputs, max_new_tokens=max_new_tokens)
    caption = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)

    return caption


In [ ]:
# Example usage
image_path = "/content/drive/MyDrive/images/blah1.jpeg"
caption = generate_image_caption(image_path, model, processor, tokenizer)
print("Generated Caption:", caption)


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Generated Caption: Describe the  a close up of a lizard. Tags: lizard, colorful, green, yellow, pink, purple, blue, red, orange, white, flower, petal, eye, mouth, tongue, chin, arm, leg, tail, color, pattern, texture, light, shadow, focus, depth, contrast, vibrancy, beauty, art, photography, nature, wildlife, exotic, tropical, jungle, habitat, environment,


In [ ]:
# Example usage
image_path = "/content/drive/MyDrive/images/girleat.jpeg"
caption = generate_image_caption(image_path, model, processor, tokenizer)
print("Generated Caption:", caption)


Generated Caption: Describe the 

The image shows a young girl eating spaghetti with a fork. She is sitting at a dining table, and the spaghetti is placed in a bowl in front of her. The girl appears to be enjoying her meal, and the scene captures a moment of her daily life.


In [ ]:
# Example usage
image_path = "/content/drive/MyDrive/images/tree.jpeg"
caption = generate_image_caption(image_path, model, processor, tokenizer)
print("Generated Caption:", caption)

Generated Caption: Describe the  the sky is blue.3

The image features a beautiful tree with a vibrant pink and red color, standing in a field. The tree is surrounded by a lush green field, and it appears to be the focal point of the scene. The sky in the background is a deep blue, adding to the serene atmosphere of the image.


In [ ]:
# Example usage
image_path = "/content/drive/MyDrive/images/boy2.jpeg"
caption = generate_image_caption(image_path, model, processor, tokenizer)
print("Generated Caption:", caption)

Generated Caption: Describe the  on the floor.

The boy is wearing headphones and playing a video game. He is sitting on the floor and holding a game controller. The boy is wearing a blue shirt and black pants.


code for anvil linking and uploading image from app:

In [6]:
pip install anvil-uplink

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.5 MB/s eta 0:00:00


In [7]:
import anvil.server

anvil.server.connect("server_UUTW3ZUBKUIIEBHCQCKXXCXO-WCJR7H4SLBMEYABK")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [8]:
!pip install pyngrok


In [9]:
!ngrok config add-authtoken 2rzVVTqhwrctWfm4cz9pNzJ1ATl_5bvWRbEtu66yTDXK3eS4w

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
from pyngrok import ngrok

# Start ngrok to expose the Flask app
public_url = ngrok.connect(8000)  # 5000 is the port Flask is using
print(f"Your ngrok URL is: {public_url}")


Your ngrok URL is: NgrokTunnel: "https://80cc-34-143-252-162.ngrok-free.app" -> "http://localhost:8000"


In [11]:
max_new_tokens=100


In [12]:
from PIL import Image
import torch
from io import BytesIO

model_id = "llava-hf/llava-1.5-7b-hf"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

LLAVA_CHAT_TEMPLATE = """A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. {% for message in messages %}{% if message['role'] == 'user' %}USER: {% else %}ASSISTANT: {% endif %}{% for item in message['content'] %}{% if item['type'] == 'text' %}{{ item['text'] }}{% elif item['type'] == 'image' %}<image>{% endif %}{% endfor %}{% if message['role'] == 'user' %} {% else %}{{eos_token}}{% endif %}{% endfor %}"""

global_model = LlavaForConditionalGeneration.from_pretrained(model_id,
                                                              quantization_config=quantization_config,
                                                              torch_dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.chat_template = LLAVA_CHAT_TEMPLATE

processor = AutoProcessor.from_pretrained(model_id)
processor.tokenizer = tokenizer
@anvil.server.callable
def generate_image_caption(uploaded_file):
    try:
        # Load and validate the uploaded image
        image_bytes = uploaded_file.get_bytes()
        image = Image.open(BytesIO(image_bytes)).convert("RGB")

        # Prepare text prompt
        text_prompt = "Describe the <image>"
        processed_inputs = processor(images=image, text=text_prompt, return_tensors="pt")

        # Add <image> token to tokenizer if not already present
        if "<image>" not in tokenizer.get_vocab():
            tokenizer.add_tokens(["<image>"], special_tokens=True)
            global_model.resize_token_embeddings(len(tokenizer))
        processor.tokenizer = tokenizer

        # Move model and inputs to appropriate device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        global_model.to(device)
        processed_inputs = {k: v.to(device) for k, v in processed_inputs.items()}

        # Generate caption
        outputs = global_model.generate(**processed_inputs, max_new_tokens=100)
        caption = processor.tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

        return caption

    except Exception as e:
        # Log traceback for debugging
        import traceback
        traceback.print_exc()
        return f"Error generating caption: {str(e)}"








/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


anvil password: itsme@Anvil22
